# NSIDC Iceflow example

This notebook shows an example of how to use the `nsidc-iceflow` Python library to do ITRF transformations with real data. We recommend starting with the [corrections.ipynb](https://github.com/nsidc/NSIDC-Data-Tutorials/blob/main/notebooks/iceflow/corrections.ipynb) notebook to learn more about ITRF transformations and why they matter.
## Finding, downloading, and reading data

Lets assume we want to do an analysis using [IceBridge ATM L1B Qfit Elevation and Return Strength, Version 1 (ILATM1B)](https://nsidc.org/data/ilatm1b/versions/1) data near Pine Island Glacier in Antarctica.

Finding, downloading, and reading ILATM1B v1 data with `nsidc-iceflow` is straightforward. 

In [ ]:
from pathlib import Path
import datetime as dt

from nsidc.iceflow.api import fetch_iceflow_df
from nsidc.iceflow.data.models import DatasetSearchParameters, BoundingBox, ATM1BDataset

# Downloaded data will go here.
data_path = Path("./downloaded-data/")

# Define a bounding box for our area of interest. Note that this is a very small area near Pine Island Glacier.
# ILATM1B data can be very large, so for the purposes of this example we will focus on just a small area with a manageable amount of data.
BBOX = BoundingBox(lower_left_lon=-103.125559, lower_left_lat=-75.180563, upper_right_lon=-102.677327, upper_right_lat=-74.798063)

# Define the dataset that we want to search for. ATM1B version 1 is the only version available at the time of writing.
atm1b_v1_dataset = ATM1BDataset(version="1")

# We will define a short date range in 2009 to search for data. Again, we choose this primarily to keep the amount of data manageable.
date_range = (dt.date(2009, 11, 1), dt.date(2009, 12, 31))

# Now use the `fetch_iceflow_df` function to search for and download data matching our search parameters. The output is a pandas DataFrame containing the matching data.
iceflow_df = fetch_iceflow_df(
    dataset_search_params=DatasetSearchParameters(
        dataset=atm1b_v1_dataset,
        bounding_box=BBOX,
        temporal=date_range,
    ),
    output_dir=data_path,
    output_itrf=None,
)

iceflow_df

For the purposes of the rest of this example, we will narrow our focus to just the latitdue, longitude, and elevation, and the ITRF data

In [ ]:
iceflow_df = iceflow_df[["latitude", "longitude", "elevation", "ITRF"]]
iceflow_df

## ITRF transformations

Lets say we have other data that are in ITRF2014, and we want to compare the ILATM1B v1 data we just fetched to it. `nsidc-iceflow` provides a `transform_itrf` function that allows the user to transform latitude, longitude, and elevation data from an `nsidc-iceflow` dataframe into a target ITRF.

In [ ]:
from nsidc.iceflow.itrf.converter import transform_itrf

itrf2014_df = transform_itrf(data=iceflow_df, target_itrf="ITRF2014")
itrf2014_df

Let's take a look at the differences between the original data (ITRF2005) and the data that has been transformed to ITRF2014.

In [ ]:
for variable in ["latitude", "longitude", "elevation"]:
    print(f"Max difference in {variable}: {abs(itrf2014_df[variable] - iceflow_df[variable]).max()}")

We can see here that the differences are very small! The largest elevation difference is 0.007 - 7mm!

## Propagating data to a new epoch

Now lets assume we need to propagate the data to a target epoch of 2019.7, which corresponds to September 13, 2019. This accounts for continental plate motion.

The `transform_itrf` function optionally takes a `target_epoch` in order to do this transformation, which we'll use the original data as input

In [ ]:
itrf2014_epoch_2019_7_df = transform_itrf(
    data=iceflow_df,
    target_itrf="ITRF2014",
    target_epoch="2019.7",
)

itrf2014_epoch_2019_7_df

The maximum difference is still small:

In [ ]:
for variable in ["latitude", "longitude", "elevation"]:
    print(f"Max difference in {variable}: {abs(itrf2014_epoch_2019_7_df[variable] - iceflow_df[variable]).max()}")

## Visualizing the differences

To visualize the differences, which are very small, we need to zoom in on just a small subset of points.

In [ ]:
filter_condition = (iceflow_df.reset_index().index > 50) & ( iceflow_df.reset_index().index < 60)
sampled_iceflow_df = iceflow_df[filter_condition]
sampled_itrf2014_df = itrf2014_df[filter_condition]
sampled_itrf2014_epoch_2019_7_df = itrf2014_epoch_2019_7_df[filter_condition]

In [ ]:
import matplotlib.pyplot as plt
# note: use `inline` to save the resulting image as an embedded png (nice for sharing). \
# Use `widget` to obtain interactive controls to explore the data in depth!
%matplotlib inline
# %matplotlib widget


fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection="3d")
ax.scatter(
    sampled_iceflow_df.longitude.values,
    sampled_iceflow_df.latitude.values,
    sampled_iceflow_df.elevation.values,
    color="blue",
    marker="o",
    label="original",
)

ax.scatter(
    sampled_itrf2014_df.longitude.values,
    sampled_itrf2014_df.latitude.values,
    sampled_itrf2014_df.elevation.values,
    color="green",
    marker="x",
    label="ITRF2014",
)

ax.scatter(
    sampled_itrf2014_epoch_2019_7_df.longitude.values,
    sampled_itrf2014_epoch_2019_7_df.latitude.values,
    sampled_itrf2014_epoch_2019_7_df.elevation.values,
    color="red",
    marker="v",
    label="ITRF2014 epoch 2019.7",
)

ax.set_xlabel("longitude (degrees)", labelpad=10)
ax.set_ylabel("latitude (degrees)", labelpad=10)
ax.set_zlabel("elevation (m)", labelpad=10)
plt.legend(loc="upper left")
plt.show()